In [29]:
import numpy as np
import random

PERIOD_LENGTH = 6 # numero de horas de cada periodo
N_PERIODS_YEAR = int((24/PERIOD_LENGTH) * 365) # numero de periodos per year

# conjuntos
sources = ['solar', 'wind', 'hydroelectric', 'nuclear', 'gas', 'coal', 'oil'] # conjunto _i
periods = [i for i in range(N_PERIODS_YEAR)] # conjunto _t

# parametros constantes
budget = 1000000 # presupuesto en dolares

# TO-DOO
# demanda por periodo en kWh D_t
demand = {period:demand for period, demand in 
                  zip(periods, np.array([random.randint(100, 1000) for _ in range(len(periods))]))}

battery_cost = 103240 # costo por bateria en dolares AC
battery_capacity = 232 # capacidad de almacenamiento en kWh por bateria AQ
battery_efficiency = 0.87 # eficiencia de energia de las baterias AE
pollution_per_battery = 46400 # kgs de CO2 emitidos por bateria producida AZ

kwh_price_mean = 0.1 # valor por kWh en el mercado promedio en dolares
kwh_price_chile = 0.16 # valor por kWh en chile en dolares


# parametro CI_i costo inicial por kW de capacidad
overnight_cost = {
    'solar': [1331, 1313, 1100],
    'wind': [1100, 1500, 1319],
    'hydroelectric': [5316, 1750, 7500, 3500],
    'nuclear': [6041, 6317, 6900, 12000],
    'gas': [950, 713, 1084, 1300],
    'coal': [3000, 6250, 5876, 3676],
    'oil': [1170, 1175, 713],
    }

# modelaremos ocupando la media en casos donde hayan mas de un dato
overnight_cost_mean = {
    key:np.mean(value) for key, value in  zip(overnight_cost.keys(), 
                                              overnight_cost.values())
}
    
# parametro C_i costo de produccion por kWh
production_cost = {
    'solar': [0.17, 0.136],
    'wind': [0.3, 0.32],
    'hydroelectric': 0.5,
    'nuclear': [2.10, 1.38, 1.6],
    'gas': [2.1],
    'coal': [0.98, 1.4, 1.73],
    'oil': [5.4, 7.01, 5.63],
}

# al igual que el costo inicial, ocuparemos la media
production_cost_mean = {
    key:np.mean(value) for key, value in zip(production_cost.keys(),
                                            production_cost.values())
}

# kg de CO2 emitidos por producir un kWh Z_i
kg_CO2_per_kwh = {
    'solar': 0.085,
    'wind': 0.026,
    'hydroelectric': 0.026,
    'nuclear': 0.029,
    'gas': 0.499,
    'coal': 0.888,
    'oil': 0.733,
    }

# to-do: cantidad maxima disponible para instalaciones QM_i
prod_units_limit = {source:limit for source, limit in zip(sources, np.array([random.randint(100, 2000) for _ in
                                                                            range(len(sources))]))}

In [30]:
from gurobipy import GRB, Model, quicksum

model = Model('Energy production planning')

prod_units = model.addVars(sources, vtype=GRB.INTEGER, name='prod_units')
storage = model.addVar(vtype=GRB.INTEGER, name='storage')
output = model.addVars(periods, sources, vtype=GRB.CONTINUOUS, name='output')
stored = model.addVars(periods, vtype=GRB.CONTINUOUS, name='stored')

model.update()

In [31]:
# restriccion 1: inversion inicial menor o igual a presupuesto
model.addConstr(quicksum(prod_units[source] * overnight_cost_mean[source]
                          for source in sources) <= budget)

# restriccion 2: recuperacion de inversion en 1 ano
# model.addConstr((quicksum(output[sources, periods] * (kwh_price_mean - )))

# restriccion 3: satisfaccion de demanda y flujo
model.addConstrs(stored[period] == quicksum(output[period, source] - demand[period] + \
                                            (stored[period - 1] * battery_efficiency) for source in sources) \
                                                for idx, period in enumerate(periods) if idx != 0)

# restriccion 4: produccion de energia limitada
model.addConstrs(output[period, source] <= prod_units[source] * PERIOD_LENGTH for source in sources
                                                                                for period in periods)
# restriccion 5: cantidad de instalaciones limitadas 
model.addConstrs(prod_units[source] <= prod_units_limit[source] for source in sources)

# restriccion 6: cantidad maxima de almacenamiento
model.addConstrs(stored[period] <= storage * battery_capacity for period in periods)


# Naturaleza de las variables
# restriccion 7:
model.addConstrs(output[period, source] >= 0 for source in sources for period in periods)
# restriccion 8:
model.addConstrs(prod_units[source] >= 0 for source in sources)
# restriccion 9:
model.addConstr(storage >= 0)
# restriccion 10:
model.addConstrs(stored[period] >= 0 for idx, period in enumerate(periods) if idx != 0)
model.addConstrs(stored[period] == 0 for idx, period in enumerate(periods) if idx == 0)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>}

In [32]:
obj = quicksum(output[period, source] * kg_CO2_per_kwh[source]\
               for source in sources for period in periods) + storage * pollution_per_battery

In [34]:
model.setObjective(obj, GRB.MINIMIZE)

model.optimize()

model.printAttr('X')
print('\n---------------------\n')
for constr in model.getConstrs():
    print(constr, constr.getAttr('slack'))
    
model.write('model.sol')

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 24835 rows, 11688 columns and 48193 nonzeros
Model fingerprint: 0x68283ad7
Variable types: 11680 continuous, 8 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+03]
  Objective range  [3e-02, 5e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 1e+06]

MIP start from previous solve produced solution with objective 155715 (0.02s)
Loaded MIP start from previous solve with objective 155715

Presolve removed 11703 rows and 8 columns
Presolve time: 0.02s
Presolved: 13132 rows, 11680 columns, 36481 nonzeros
Variable types: 11672 continuous, 8 integer (0 binary)

Root relaxation: objective 1.556851e+05, 3047 iterations, 0.31 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 155697.906    0    2 155714.618 155697.906  0.01%     -    0s
     0     0 155698.19

output[222,wind]      1495.24 
output[223,wind]      1656.24 
output[224,wind]        25.24 
output[225,wind]       893.24 
output[226,wind]      2776.24 
output[227,solar]        56.24 
output[227,wind]         3930 
output[227,hydroelectric]           36 
output[228,wind]       998.81 
output[230,wind]      2086.51 
output[235,wind]      1950.24 
output[236,wind]      3462.24 
output[237,wind]       158.24 
output[238,wind]      1684.24 
output[239,wind]      2272.24 
output[240,wind]      1173.24 
output[241,wind]      2401.52 
output[243,wind]      1117.24 
output[244,wind]      3609.24 
output[245,wind]      1838.24 
output[246,wind]      2839.24 
output[247,wind]      3112.24 
output[248,wind]      3539.24 
output[249,wind]      3749.24 
output[250,wind]      2720.24 
output[251,wind]      3112.24 
output[252,wind]      2153.24 
output[253,wind]      3098.24 
output[254,wind]      2131.24 
output[256,wind]      3570.54 
output[259,wind]      1940.15 
output[261,wind]      3770.24

output[532,wind]      2321.24 
output[533,solar]      428.867 
output[533,wind]         3930 
output[533,hydroelectric]           36 
output[535,wind]      1348.24 
output[536,wind]       424.24 
output[537,wind]       137.24 
output[538,wind]      1628.24 
output[539,solar]       539.24 
output[539,wind]         3930 
output[539,hydroelectric]           36 
output[540,wind]      1053.55 
output[542,wind]      2076.24 
output[543,solar]       462.24 
output[543,wind]         3930 
output[543,hydroelectric]           36 
output[544,wind]       872.24 
output[545,wind]      1803.24 
output[546,wind]      2215.98 
output[548,wind]       578.24 
output[549,wind]      1697.98 
output[552,solar]       455.24 
output[552,wind]         3930 
output[552,hydroelectric]           36 
output[553,wind]      3769.24 
output[553,hydroelectric]           36 
output[554,wind]      2265.24 
output[555,wind]      3343.24 
output[556,wind]      2132.24 
output[557,solar]       392.24 
output[557,wind]    

output[801,hydroelectric]           36 
output[802,wind]      3757.99 
output[802,hydroelectric]           36 
output[804,wind]      2258.24 
output[805,wind]      3909.24 
output[805,hydroelectric]           36 
output[806,wind]      3575.95 
output[806,hydroelectric]           36 
output[810,wind]      1406.46 
output[812,wind]      2695.51 
output[814,wind]      2489.24 
output[815,wind]        67.24 
output[816,wind]      3770.24 
output[817,solar]       315.24 
output[817,wind]         3930 
output[817,hydroelectric]           36 
output[818,wind]      2468.24 
output[819,wind]       697.24 
output[820,wind]      965.695 
output[822,wind]      3805.24 
output[823,wind]      2195.24 
output[824,wind]      3217.24 
output[825,solar]          558 
output[825,wind]         3930 
output[825,hydroelectric]           36 
output[826,wind]      1801.18 
output[826,hydroelectric]           36 
output[827,wind]      3524.24 
output[827,hydroelectric]           36 
output[828,wind]      1719.

output[1065,solar]          558 
output[1065,wind]         3930 
output[1065,hydroelectric]           36 
output[1066,wind]      2835.01 
output[1066,hydroelectric]           36 
output[1068,wind]      2083.24 
output[1069,wind]      2565.25 
output[1071,wind]      784.696 
output[1076,wind]      3567.24 
output[1077,solar]       483.24 
output[1077,wind]         3930 
output[1077,hydroelectric]           36 
output[1078,solar]      29.9707 
output[1078,wind]         3930 
output[1078,hydroelectric]           36 
output[1080,wind]       445.24 
output[1081,wind]      3490.24 
output[1082,wind]       949.24 
output[1083,wind]       935.24 
output[1084,wind]      3511.24 
output[1085,wind]      3658.24 
output[1086,solar]      232.959 
output[1086,wind]         3930 
output[1086,hydroelectric]           36 
output[1088,wind]      35.6052 
output[1091,wind]      2762.24 
output[1092,wind]       837.24 
output[1093,solar]       490.24 
output[1093,wind]         3930 
output[1093,hydroelect

...

<gurobi.Constr R12682> 104.8195620697744
<gurobi.Constr R12683> 124.5911330049259
<gurobi.Constr R12684> -1.2505552149377763e-12
<gurobi.Constr R12685> -1.1368683772161603e-13
<gurobi.Constr R12686> -7.958078640513122e-13
<gurobi.Constr R12687> 53.326765188834145
<gurobi.Constr R12688> -1.1368683772161603e-13
<gurobi.Constr R12689> 176.3152709359606
<gurobi.Constr R12690> -1.1368683772161603e-13
<gurobi.Constr R12691> 86.66009852216723
<gurobi.Constr R12692> -1.5347723092418164e-12
<gurobi.Constr R12693> -1.1368683772161603e-13
<gurobi.Constr R12694> -1.1368683772161603e-13
<gurobi.Constr R12695> -1.1368683772161603e-13
<gurobi.Constr R12696> -1.1368683772161603e-13
<gurobi.Constr R12697> 183.2118226600985
<gurobi.Constr R12698> -1.1368683772161603e-13
<gurobi.Constr R12699> -1.1368683772161603e-13
<gurobi.Constr R12700> 0.0
<gurobi.Constr R12701> 0.453201970443331
<gurobi.Constr R12702> -1.1368683772161603e-13
<gurobi.Constr R12703> -1.1368683772161603e-13
<gurobi.Constr R12704> 254.4

<gurobi.Constr R12909> -1.1368683772161603e-13
<gurobi.Constr R12910> -1.1368683772161603e-13
<gurobi.Constr R12911> -1.1937117960769683e-12
<gurobi.Constr R12912> -1.1368683772161603e-13
<gurobi.Constr R12913> 47.57963875205252
<gurobi.Constr R12914> -1.1368683772161603e-13
<gurobi.Constr R12915> -3.410605131648481e-13
<gurobi.Constr R12916> -2.0463630789890885e-12
<gurobi.Constr R12917> -1.1368683772161603e-12
<gurobi.Constr R12918> -1.1368683772161603e-13
<gurobi.Constr R12919> -1.1368683772161603e-12
<gurobi.Constr R12920> 221.14285714285714
<gurobi.Constr R12921> -1.1368683772161603e-13
<gurobi.Constr R12922> -1.1368683772161603e-13
<gurobi.Constr R12923> -7.958078640513122e-13
<gurobi.Constr R12924> -1.1368683772161603e-13
<gurobi.Constr R12925> -1.1368683772161603e-13
<gurobi.Constr R12926> -1.1368683772161603e-12
<gurobi.Constr R12927> 231.25823107681438
<gurobi.Constr R12928> 201.60262725779967
<gurobi.Constr R12929> 0.0
<gurobi.Constr R12930> 0.0
<gurobi.Constr R12931> -1.136

<gurobi.Constr R13204> -480.2580623973655
<gurobi.Constr R13205> 0.0
<gurobi.Constr R13206> 0.0
<gurobi.Constr R13207> 0.0
<gurobi.Constr R13208> 0.0
<gurobi.Constr R13209> 0.0
<gurobi.Constr R13210> 0.0
<gurobi.Constr R13211> 0.0
<gurobi.Constr R13212> 0.0
<gurobi.Constr R13213> 0.0
<gurobi.Constr R13214> 0.0
<gurobi.Constr R13215> 0.0
<gurobi.Constr R13216> 0.0
<gurobi.Constr R13217> 0.0
<gurobi.Constr R13218> 0.0
<gurobi.Constr R13219> 0.0
<gurobi.Constr R13220> 0.0
<gurobi.Constr R13221> 0.0
<gurobi.Constr R13222> -246.03967159277522
<gurobi.Constr R13223> 0.0
<gurobi.Constr R13224> 0.0
<gurobi.Constr R13225> 0.0
<gurobi.Constr R13226> 0.0
<gurobi.Constr R13227> 0.0
<gurobi.Constr R13228> 0.0
<gurobi.Constr R13229> 0.0
<gurobi.Constr R13230> 0.0
<gurobi.Constr R13231> 0.0
<gurobi.Constr R13232> 0.0
<gurobi.Constr R13233> -224.24000000005023
<gurobi.Constr R13234> 0.0
<gurobi.Constr R13235> -14.239999999999972
<gurobi.Constr R13236> 0.0
<gurobi.Constr R13237> 0.0
<gurobi.Constr R132

<gurobi.Constr R13530> 0.0
<gurobi.Constr R13531> 0.0
<gurobi.Constr R13532> 0.0
<gurobi.Constr R13533> 0.0
<gurobi.Constr R13534> 0.0
<gurobi.Constr R13535> 0.0
<gurobi.Constr R13536> 0.0
<gurobi.Constr R13537> 0.0
<gurobi.Constr R13538> 0.0
<gurobi.Constr R13539> -385.2399999999917
<gurobi.Constr R13540> 0.0
<gurobi.Constr R13541> 0.0
<gurobi.Constr R13542> 0.0
<gurobi.Constr R13543> 0.0
<gurobi.Constr R13544> 0.0
<gurobi.Constr R13545> 0.0
<gurobi.Constr R13546> 0.0
<gurobi.Constr R13547> 0.0
<gurobi.Constr R13548> 0.0
<gurobi.Constr R13549> 0.0
<gurobi.Constr R13550> 0.0
<gurobi.Constr R13551> 0.0
<gurobi.Constr R13552> 0.0
<gurobi.Constr R13553> 0.0
<gurobi.Constr R13554> 0.0
<gurobi.Constr R13555> -322.2399999999917
<gurobi.Constr R13556> 0.0
<gurobi.Constr R13557> 0.0
<gurobi.Constr R13558> 0.0
<gurobi.Constr R13559> 0.0
<gurobi.Constr R13560> 0.0
<gurobi.Constr R13561> 0.0
<gurobi.Constr R13562> 0.0
<gurobi.Constr R13563> 0.0
<gurobi.Constr R13564> 0.0
<gurobi.Constr R13565> -3

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
model.computeIIS()
model.write("model.ilp")



Computing Irreducible Inconsistent Subsystem (IIS)...

      Constraints           Bounds       Runtime
     Min       Max       Min      Max
------------------------------------------------
        0    24835         0    11688         0s
       24       24         1        1         0s

IIS computed: 24 constraints, 1 bounds
IIS runtime: 0.24 seconds


In [12]:
if model.status == GRB.INFEASIBLE:
    model.feasRelaxS(1, False, False, True)
    model.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 24835 rows, 37983 columns and 74488 nonzeros
Model fingerprint: 0x3eea65a5
Model has 26295 quadratic objective terms
Variable types: 37975 continuous, 8 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+03]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+08]

MIP start from previous solve produced solution with objective 7.40088e+08 (0.49s)
Loaded MIP start from previous solve with objective 7.40088e+08

Presolve removed 13161 rows and 13171 columns
Presolve time: 0.03s
Presolved: 11674 rows, 24812 columns, 46697 nonzeros
Presolved model has 13140 quadratic objective terms
Found heuristic solution: objective 1.693091e+08
Variable types: 24805 continuous, 7 integer (0 binary)

Root relaxation: objective 9.637222e+03, 3016 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds 